In [1]:
import numpy as np
from collections import deque

def string_field():
    print("-" * len(field[0]))
    print(round_played)
    uns = sorted(units)
    for i in range(len(field)):
        for j in range(len(field[i])):
            if (i, j) in units_location:
                print("E" if units_location[(i, j)].is_elve else "G", end="")
            else:
                print(field[i][j], end="")
        print(" " * 3, end="")
        for un in uns:
            if un.i == i and un.hp > 0:
                print("E" if un.is_elve else "G", end="")
                print("(", un.hp, ")", sep="", end=" ")
        print()
    print("-" * len(field[0]))


class Unit:

    def __init__(self, i, j, who):
        self.i = i
        self.j = j
        if who == "G":
            self.is_elve = False
        elif who == "E":
            self.is_elve = True
        self.hp = 200

    def __lt__(self, other):
        assert isinstance(other, Unit)
        if self.i == other.i:
            return self.j < other.j
        else:
            return self.i < other.i

    def attack(self, other):
        assert isinstance(other, Unit)
        other.hp -= 3

    @property
    def ij(self):
        return self.i, self.j

    def __repr__(self):
        return "{" + ', '.join(map(str, [self.i, self.j, self.is_elve, self.hp])) + "}"


def bfs(start):
    found_units = []
    deq = deque()
    deq.append(start.ij)
    came_from = {start.ij: None}
    used = set()
    path = []
    found = False
    while len(deq) != 0:
        i, j = deq.popleft()

        path.append((i, j))
        if field[i][j] == "#" or (i, j) in used:
            continue

        used.add((i, j))
        if (i, j) in units_location and (i, j) != start.ij and units_location[(i, j)].hp > 0:
            unit = units_location[(i, j)]
            if unit.is_elve ^ start.is_elve:
                found_units.append(units_location[(i, j)])
                found = True
            continue

        if not found:
            for adj in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
                cur = i + adj[0], j + adj[1]
                if cur not in came_from:
                    came_from[cur] = (i, j)
                    deq.append(cur)

    # Get the closest in reading order
    nearest_step = None
    nearest_path = None
    nearest_last_step = None
    for unit in found_units:
        un = unit.ij
        next_step = un
        path_len = 1
        while came_from[un]:
            next_step = un
            un = came_from[un]
            path_len += 1
        to_step = next_step if path_len > 2 else start.ij
        last_step = came_from[unit.ij]
        if not nearest_step or (is_smaller(last_step, nearest_last_step) and path_len == nearest_path):
            nearest_step = to_step
            nearest_path = path_len
            nearest_last_step = last_step
    return nearest_step


def is_smaller(step1, step2):
    if step1[0] == step2[0]:
        return step1[1] < step2[1]
    else:
        return step1[0] < step2[0]


def step(unit):
    to_step = bfs(unit)
    if not to_step:
        return
    assert all([units_location[un].hp > -1 for un in units_location]), "Ne norm"
    units_location.pop(unit.ij)
    unit.i = to_step[0]
    unit.j = to_step[1]
    units_location[to_step] = unit


def attack_near(unit):
    global elves, gnomes
    i, j = unit.ij
    lowest_hp = None
    unit_to_attack = None
    for adj in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
        near = i + adj[0], j + adj[1]
        if near in units_location and units_location[near].is_elve ^ unit.is_elve:
            near_unit = units_location[near]
            if not lowest_hp or near_unit.hp < lowest_hp:
                lowest_hp = near_unit.hp
                unit_to_attack = near_unit
    if unit_to_attack:
        unit.attack(unit_to_attack)
        if unit_to_attack.hp <= 0:
            units_location.pop(unit_to_attack.ij)
            if unit_to_attack.is_elve:
                elves -= 1
            else:
                gnomes -= 1
        assert all([units_location[un].hp > -1 for un in units_location])


def played_full_round():
    global units
    unit_left_to_step_after_killing_the_last = False
    finished = False
    for unit in sorted(units):
        if unit.hp <= 0:
            continue
        step(unit)
        if finished:
            unit_left_to_step_after_killing_the_last = True
            break
        attack_near(unit)
        if (elves == 0 or gnomes == 0) and not finished:
            unit_left_to_step_after_killing_the_last = False
            finished = True

    units = [units_location[pos] for pos in units_location]

    if finished:
        return int(not unit_left_to_step_after_killing_the_last)
    else:
        return 1


def calc_hp_left():
    hp_left = 0
    for unit_pos in units_location:
        hp_left += units_location[unit_pos].hp
    return hp_left

In [2]:
field = []
units = []
units_location = {}
elves, gnomes = 0, 0
with open("input.txt") as inp:
    for i, line in enumerate(map(lambda x: list(x.rstrip()), inp.readlines())):
        for j, elem in enumerate(line):
            if elem in "EG":
                units.append(Unit(i, j, elem))
                units_location[(i, j)] = units[-1]
                if units[-1].is_elve:
                    elves += 1
                else:
                    gnomes += 1
                line[j] = "."
        field.append(line)
field = np.array(field)


round_played = 0
while True:
    round_played += played_full_round()
    if elves == 0 or gnomes == 0:
        break
string_field()


hp_left = calc_hp_left()
print(round_played, hp_left)
print(hp_left * round_played)

--------------------------------
103
################################   
#######.....####################   
#########...####################   
#########...####################   
#########.######################   
#########.######################   
#########.######################   
#########.#...##################   
#########.....#..###############   
########......G.###.....########   G(200) 
#######.........G.......########   G(98) 
#######...GG..G........#########   G(200) G(200) G(200) 
######........#####......#######   
######.......#######......######   
#####.......#########.G.G....###   G(110) G(200) 
#####.####..#########..G.#....##   G(68) 
####..####..#########...G......#   G(200) 
#####.####..#########...G.....##   G(200) 
#########...#########.G.......##   G(194) 
#####........#######.G.G......##   G(146) G(200) 
######.......G#####...##...#..##   G(200) 
###...................####.##.##   
###.............#########..#####   
#.#.#...........#########..#####   
#..